In [11]:
import os
import sys
import math
import numpy as np
import netCDF4 as nc
from matplotlib import pyplot as plt
from math import exp, sqrt, log
import time
import geopandas as gpd
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
if '..' not in sys.path:
    sys.path.append('..')
from importlib import reload
reload(util_tools)
import util_tools
from util_tools.data_loader import data_processer

# 1. Data Processing

In [84]:
# define necessary parameters
target_var = 'DUEXTTAU'
file_path_g_06 = r'C:\Users\96349\Documents\Downscale_data\MERRA2\G5NR_aerosol_variables_over_MiddleEast_daily_20060516-20070515.nc'
file_path_g_05 = r'C:\Users\96349\Documents\Downscale_data\MERRA2\G5NR_aerosol_variables_over_MiddleEast_daily_20050516-20060515.nc'
file_path_m = r'C:\Users\96349\Documents\Downscale_data\MERRA2\MERRA2_aerosol_variables_over_MiddleEast_daily_20000516-20180515.nc'
file_path_ele = r'C:\Users\96349\Documents\Downscale_data\elevation\elevation_data.npy'
file_path_country = [r'C:\Users\96349\Documents\Downscale_data\Country_shape\AFG_adm/AFG_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\ARE_adm/ARE_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\IRQ_adm/IRQ_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\KWT_adm/KWT_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\QAT_adm/QAT_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\SAU_adm/SAU_adm0.shp']
n_lag = 10
n_pred = 3
task_dim = [5, 5]

In [85]:
reload(util_tools.data_loader)
import util_tools.data_loader as data_loader

In [86]:
data_processor = data_loader.data_processer()
g_data, m_data, [G_lats, G_lons, M_lats, M_lons], ele_data = data_processor.load_data(target_var, file_path_g_05, file_path_g_06, file_path_m, file_path_ele, file_path_country)

In [91]:
m_data = data_processor.unify_m_data(g_data[:10], m_data, G_lats, G_lons, M_lats, M_lons)

In [92]:
match_m_data = m_data[range(1826, 1826+730), :, :]

In [93]:
match_m_data.shape == (730, 123, 207)

True

In [112]:
X_high, X_low, X_ele, X_other, Y = data_processor.flatten(g_data[:50, :60, :100], match_m_data[:50, :60, :100], ele_data[:60, :100],
                                                          [G_lats[:60], G_lons[:100]], list(range(50)), n_lag=n_lag, n_pred=n_pred, task_dim=task_dim, is_perm=True, 
                                                          return_Y=True)

# 2. GAN model test

In [144]:
def nnelu(input):
    return tf.add(tf.constant(1, dtype=tf.float32), tf.nn.elu(input))

def mapping_to_target_range( x, target_min=0, target_max=1 ) :
    x02 = tf.keras.backend.tanh(x) + 1 # x in range(0,2)
    scale = ( target_max-target_min )/2.
    return  x02 * scale + target_min


high_input = tf.keras.Input(shape=(n_lag, task_dim[0], task_dim[1], 1))
x1 = tf.keras.layers.ConvLSTM2D(16, kernel_size=(3,3), return_sequences=True, activation=tf.keras.layers.LeakyReLU())(high_input)
x1 = tf.keras.layers.ConvLSTM2D(16, kernel_size=(3,3), activation=tf.keras.layers.LeakyReLU())(x1)
x1 = tf.keras.layers.Flatten()(x1)

low_input = tf.keras.Input(shape=(n_lag, task_dim[0], task_dim[1], 1))
x2 = tf.keras.layers.ConvLSTM2D(16, kernel_size=(3,3), activation=tf.keras.layers.LeakyReLU())(low_input)
x2 = tf.keras.layers.Flatten()(x1)

ele_input = tf.keras.Input(shape=(task_dim[0], task_dim[1], 1))
x3 = tf.keras.layers.Conv2D(16, kernel_size=(3,3), activation=tf.keras.layers.LeakyReLU())(ele_input)
x3 = tf.keras.layers.Flatten()(x3)

other_input =  tf.keras.Input(shape=(3))
x4 = tf.keras.layers.Dense(8, activation=tf.keras.layers.LeakyReLU())(other_input)

x = tf.keras.layers.Concatenate(axis=1)([x1, x2, x3, x4])
#x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(30, kernel_initializer="he_normal", use_bias=True, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(n_pred*np.prod(task_dim), activation=nnelu)(x)
x = tf.keras.layers.Reshape([n_pred, task_dim[0], task_dim[1]])(x)
generator = tf.keras.Model([high_input, low_input, ele_input, other_input], x)
opt = tf.keras.optimizers.Adam(learning_rate=0.005)
generator.compile(optimizer=opt, loss='mean_absolute_error')

In [145]:
# define callbacks
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
best_save = tf.keras.callbacks.ModelCheckpoint('s2s_model', save_best_only=True, monitor='val_loss', mode='min')
callbacks = [lr_scheduler, early_stopping, best_save]

history = generator.fit([X_high[:-3], X_low[:-3], X_ele[:-3], X_other[:-3]], Y[:-3], epochs=1, callbacks=callbacks, validation_split=0.25)

4788/4788 [==============================] - ETA: 0s - loss: 0.0091

INFO:tensorflow:Assets written to: s2s_model\assets


INFO:tensorflow:Assets written to: s2s_model\assets


4788/4788 [==============================] - 152s 31ms/step - loss: 0.0091 - val_loss: 0.0070 - lr: 0.0050


In [149]:
generator.predict([X_high[-2:], X_low[-2:], X_ele[-2:], X_other[-2:]])

array([[[[0.9376699 , 0.9386129 , 0.93721837, 0.9373431 , 0.93620485],
         [0.93568444, 0.9370458 , 0.93726045, 0.93531597, 0.93529063],
         [0.9378946 , 0.9349826 , 0.93719894, 0.9350653 , 0.9363637 ],
         [0.93569696, 0.9353592 , 0.9351049 , 0.9366794 , 0.9354305 ],
         [0.93596244, 0.9370291 , 0.9369965 , 0.9372107 , 0.9383129 ]],

        [[0.9282891 , 0.9265894 , 0.9244624 , 0.9250286 , 0.9282641 ],
         [0.92641294, 0.9231562 , 0.928315  , 0.92851907, 0.9261082 ],
         [0.9285624 , 0.9280207 , 0.92779064, 0.9289942 , 0.9275635 ],
         [0.929107  , 0.92710096, 0.93064654, 0.92868507, 0.9267624 ],
         [0.9283209 , 0.927211  , 0.9292313 , 0.9263889 , 0.9298842 ]],

        [[0.9248564 , 0.92478716, 0.92508835, 0.92359024, 0.9242833 ],
         [0.92482024, 0.92457044, 0.9247948 , 0.9248016 , 0.92412555],
         [0.92480206, 0.92125404, 0.9243321 , 0.9209118 , 0.9245461 ],
         [0.92443264, 0.9248571 , 0.92524236, 0.92397183, 0.9246413 ],
  

In [20]:
g2_model = tf.keras.models.load_model('s2s_model')

In [27]:
pd.DataFrame(history.history).to_csv('history.csv')

In [26]:
pred_Y = g2_model.predict([X_high[-1:], X_low[-1:], X_ele[-1:], X_other[-1:]])

In [28]:
pred_Y.mean()

0.9427106

In [112]:
np.sqrt(np.mean(np.square(pred_Y[-1] - Y[-1])))

0.010288388300887755

In [100]:
from util_tools.data_processing import rsquared
rsquared(pred_Y[0, 0].reshape((25)), Y[0,0].reshape((25)))

(0.008515162367835552, 0.6608753429840621)

In [105]:
pred_input = tf.keras.Input(shape=(n_pred, task_dim[0], task_dim[1]))
y1 = tf.keras.layers.Flatten()(pred_input)
'''
condition_input = tf.keras.Input(shape=(3))
y2 = tf.keras.layers.Dense(8, activation='relu')(condition_input)
y = tf.keras.layers.Concatenate(axis=1)([y1, y2])
'''
y = tf.keras.layers.Dense(8, activation=tf.keras.layers.LeakyReLU())(y1)
y = tf.keras.layers.Dropout(0.8)(y)
y = tf.keras.layers.Dense(1, activation='sigmoid')(y)
discriminator = tf.keras.Model([pred_input], y)
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [120]:
reload(util_tools.cGAN_model)
from util_tools.cGAN_model import  Condition_GAN

In [121]:
cGAN = Condition_GAN(generator, discriminator, lr=0.000001)
cGAN.fit(1, 100, [X_high, X_low, X_ele, X_other], Y)

Epoch:1, batch:1/2042, real_loss=0.114, fake_loss=18.272, g_loss=16.256
Epoch:1, batch:2/2042, real_loss=0.000, fake_loss=16.661, g_loss=14.401
Epoch:1, batch:3/2042, real_loss=0.114, fake_loss=11.635, g_loss=12.328
Epoch:1, batch:4/2042, real_loss=0.000, fake_loss=11.696, g_loss=12.025
Epoch:1, batch:5/2042, real_loss=0.228, fake_loss=14.006, g_loss=9.906
Epoch:1, batch:6/2042, real_loss=0.001, fake_loss=5.630, g_loss=7.585
Epoch:1, batch:7/2042, real_loss=0.099, fake_loss=9.952, g_loss=6.714
Epoch:1, batch:8/2042, real_loss=0.116, fake_loss=4.317, g_loss=5.256
Epoch:1, batch:9/2042, real_loss=0.010, fake_loss=5.929, g_loss=3.980
Epoch:1, batch:10/2042, real_loss=0.504, fake_loss=2.233, g_loss=2.664
Epoch:1, batch:11/2042, real_loss=0.578, fake_loss=1.022, g_loss=2.256
Epoch:1, batch:12/2042, real_loss=0.657, fake_loss=2.589, g_loss=1.834
Epoch:1, batch:13/2042, real_loss=0.650, fake_loss=0.849, g_loss=1.521
Epoch:1, batch:14/2042, real_loss=0.696, fake_loss=1.214, g_loss=1.089
Epoch:

Epoch:1, batch:118/2042, real_loss=0.632, fake_loss=0.758, g_loss=0.742
Epoch:1, batch:119/2042, real_loss=0.713, fake_loss=0.834, g_loss=0.778
Epoch:1, batch:120/2042, real_loss=0.772, fake_loss=0.602, g_loss=0.788
Epoch:1, batch:121/2042, real_loss=0.680, fake_loss=0.657, g_loss=0.778
Epoch:1, batch:122/2042, real_loss=0.750, fake_loss=0.773, g_loss=0.751
Epoch:1, batch:123/2042, real_loss=0.682, fake_loss=0.746, g_loss=0.772
Epoch:1, batch:124/2042, real_loss=0.673, fake_loss=0.741, g_loss=0.792
Epoch:1, batch:125/2042, real_loss=0.573, fake_loss=0.958, g_loss=0.780
Epoch:1, batch:126/2042, real_loss=0.690, fake_loss=0.739, g_loss=0.756
Epoch:1, batch:127/2042, real_loss=0.738, fake_loss=0.798, g_loss=0.742
Epoch:1, batch:128/2042, real_loss=0.587, fake_loss=0.743, g_loss=0.768
Epoch:1, batch:129/2042, real_loss=0.569, fake_loss=0.586, g_loss=0.721
Epoch:1, batch:130/2042, real_loss=0.710, fake_loss=0.762, g_loss=0.725
Epoch:1, batch:131/2042, real_loss=0.770, fake_loss=0.741, g_los

Epoch:1, batch:233/2042, real_loss=0.724, fake_loss=0.707, g_loss=0.708
Epoch:1, batch:234/2042, real_loss=0.654, fake_loss=0.711, g_loss=0.680
Epoch:1, batch:235/2042, real_loss=0.727, fake_loss=0.727, g_loss=0.687
Epoch:1, batch:236/2042, real_loss=0.692, fake_loss=0.708, g_loss=0.712
Epoch:1, batch:237/2042, real_loss=0.673, fake_loss=0.759, g_loss=0.724
Epoch:1, batch:238/2042, real_loss=0.704, fake_loss=0.792, g_loss=0.731
Epoch:1, batch:239/2042, real_loss=0.727, fake_loss=0.711, g_loss=0.770
Epoch:1, batch:240/2042, real_loss=0.714, fake_loss=0.849, g_loss=0.766
Epoch:1, batch:241/2042, real_loss=0.615, fake_loss=0.656, g_loss=0.775
Epoch:1, batch:242/2042, real_loss=0.576, fake_loss=0.750, g_loss=0.784
Epoch:1, batch:243/2042, real_loss=0.678, fake_loss=0.733, g_loss=0.767
Epoch:1, batch:244/2042, real_loss=0.750, fake_loss=0.716, g_loss=0.724
Epoch:1, batch:245/2042, real_loss=0.673, fake_loss=0.749, g_loss=0.697
Epoch:1, batch:246/2042, real_loss=0.739, fake_loss=0.799, g_los

Epoch:1, batch:347/2042, real_loss=0.662, fake_loss=0.617, g_loss=0.671
Epoch:1, batch:348/2042, real_loss=0.760, fake_loss=0.682, g_loss=0.664
Epoch:1, batch:349/2042, real_loss=0.866, fake_loss=0.742, g_loss=0.658
Epoch:1, batch:350/2042, real_loss=0.601, fake_loss=0.620, g_loss=0.694
Epoch:1, batch:351/2042, real_loss=0.763, fake_loss=0.647, g_loss=0.708
Epoch:1, batch:352/2042, real_loss=0.671, fake_loss=0.749, g_loss=0.705
Epoch:1, batch:353/2042, real_loss=0.681, fake_loss=0.874, g_loss=0.731
Epoch:1, batch:354/2042, real_loss=0.733, fake_loss=0.851, g_loss=0.784
Epoch:1, batch:355/2042, real_loss=0.609, fake_loss=0.943, g_loss=0.826
Epoch:1, batch:356/2042, real_loss=0.619, fake_loss=0.726, g_loss=0.806
Epoch:1, batch:357/2042, real_loss=0.594, fake_loss=0.898, g_loss=0.839
Epoch:1, batch:358/2042, real_loss=0.517, fake_loss=0.845, g_loss=0.825
Epoch:1, batch:359/2042, real_loss=0.583, fake_loss=0.921, g_loss=0.820
Epoch:1, batch:360/2042, real_loss=0.644, fake_loss=0.830, g_los

Epoch:1, batch:461/2042, real_loss=0.681, fake_loss=0.714, g_loss=0.668
Epoch:1, batch:462/2042, real_loss=0.656, fake_loss=0.702, g_loss=0.678
Epoch:1, batch:463/2042, real_loss=0.668, fake_loss=0.639, g_loss=0.691
Epoch:1, batch:464/2042, real_loss=0.786, fake_loss=0.704, g_loss=0.706
Epoch:1, batch:465/2042, real_loss=0.739, fake_loss=0.759, g_loss=0.665
Epoch:1, batch:466/2042, real_loss=0.686, fake_loss=0.674, g_loss=0.677
Epoch:1, batch:467/2042, real_loss=0.661, fake_loss=0.645, g_loss=0.679
Epoch:1, batch:468/2042, real_loss=0.732, fake_loss=0.691, g_loss=0.651
Epoch:1, batch:469/2042, real_loss=0.688, fake_loss=0.648, g_loss=0.677
Epoch:1, batch:470/2042, real_loss=0.695, fake_loss=0.675, g_loss=0.678
Epoch:1, batch:471/2042, real_loss=0.719, fake_loss=0.636, g_loss=0.661
Epoch:1, batch:472/2042, real_loss=0.777, fake_loss=0.693, g_loss=0.618
Epoch:1, batch:473/2042, real_loss=0.670, fake_loss=0.686, g_loss=0.612
Epoch:1, batch:474/2042, real_loss=0.874, fake_loss=0.611, g_los

...

Epoch:1, batch:1490/2042, real_loss=0.701, fake_loss=0.682, g_loss=0.686
Epoch:1, batch:1491/2042, real_loss=0.694, fake_loss=0.692, g_loss=0.685
Epoch:1, batch:1492/2042, real_loss=0.693, fake_loss=0.698, g_loss=0.692
Epoch:1, batch:1493/2042, real_loss=0.751, fake_loss=0.690, g_loss=0.696
Epoch:1, batch:1494/2042, real_loss=0.693, fake_loss=0.686, g_loss=0.695
Epoch:1, batch:1495/2042, real_loss=0.697, fake_loss=0.706, g_loss=0.703
Epoch:1, batch:1496/2042, real_loss=0.632, fake_loss=0.651, g_loss=0.708
Epoch:1, batch:1497/2042, real_loss=0.695, fake_loss=0.723, g_loss=0.701
Epoch:1, batch:1498/2042, real_loss=0.671, fake_loss=0.710, g_loss=0.696
Epoch:1, batch:1499/2042, real_loss=0.660, fake_loss=0.698, g_loss=0.716
Epoch:1, batch:1500/2042, real_loss=0.721, fake_loss=0.700, g_loss=0.720
Epoch:1, batch:1501/2042, real_loss=0.635, fake_loss=0.727, g_loss=0.726
Epoch:1, batch:1502/2042, real_loss=0.627, fake_loss=0.741, g_loss=0.726
Epoch:1, batch:1503/2042, real_loss=0.687, fake_los

Epoch:1, batch:1604/2042, real_loss=0.705, fake_loss=0.711, g_loss=0.676
Epoch:1, batch:1605/2042, real_loss=0.694, fake_loss=0.676, g_loss=0.687
Epoch:1, batch:1606/2042, real_loss=0.734, fake_loss=0.653, g_loss=0.676
Epoch:1, batch:1607/2042, real_loss=0.691, fake_loss=0.712, g_loss=0.694
Epoch:1, batch:1608/2042, real_loss=0.667, fake_loss=0.704, g_loss=0.676
Epoch:1, batch:1609/2042, real_loss=0.760, fake_loss=0.690, g_loss=0.671
Epoch:1, batch:1610/2042, real_loss=0.769, fake_loss=0.650, g_loss=0.662
Epoch:1, batch:1611/2042, real_loss=0.782, fake_loss=0.699, g_loss=0.664
Epoch:1, batch:1612/2042, real_loss=0.697, fake_loss=0.667, g_loss=0.670
Epoch:1, batch:1613/2042, real_loss=0.704, fake_loss=0.642, g_loss=0.651
Epoch:1, batch:1614/2042, real_loss=0.698, fake_loss=0.643, g_loss=0.655
Epoch:1, batch:1615/2042, real_loss=0.713, fake_loss=0.610, g_loss=0.659
Epoch:1, batch:1616/2042, real_loss=0.765, fake_loss=0.670, g_loss=0.658
Epoch:1, batch:1617/2042, real_loss=0.764, fake_los

Epoch:1, batch:1718/2042, real_loss=0.688, fake_loss=0.721, g_loss=0.698
Epoch:1, batch:1719/2042, real_loss=0.685, fake_loss=0.701, g_loss=0.685
Epoch:1, batch:1720/2042, real_loss=0.755, fake_loss=0.670, g_loss=0.664
Epoch:1, batch:1721/2042, real_loss=0.684, fake_loss=0.674, g_loss=0.670
Epoch:1, batch:1722/2042, real_loss=0.711, fake_loss=0.677, g_loss=0.662
Epoch:1, batch:1723/2042, real_loss=0.723, fake_loss=0.700, g_loss=0.657
Epoch:1, batch:1724/2042, real_loss=0.818, fake_loss=0.659, g_loss=0.654
Epoch:1, batch:1725/2042, real_loss=0.697, fake_loss=0.700, g_loss=0.674
Epoch:1, batch:1726/2042, real_loss=0.707, fake_loss=0.671, g_loss=0.666
Epoch:1, batch:1727/2042, real_loss=0.673, fake_loss=0.698, g_loss=0.660
Epoch:1, batch:1728/2042, real_loss=0.765, fake_loss=0.629, g_loss=0.631
Epoch:1, batch:1729/2042, real_loss=0.726, fake_loss=0.638, g_loss=0.635
Epoch:1, batch:1730/2042, real_loss=0.789, fake_loss=0.647, g_loss=0.618
Epoch:1, batch:1731/2042, real_loss=0.725, fake_los

Epoch:1, batch:1831/2042, real_loss=0.716, fake_loss=0.681, g_loss=0.771
Epoch:1, batch:1832/2042, real_loss=0.606, fake_loss=0.740, g_loss=0.778
Epoch:1, batch:1833/2042, real_loss=0.725, fake_loss=0.705, g_loss=0.772
Epoch:1, batch:1834/2042, real_loss=0.635, fake_loss=0.854, g_loss=0.768
Epoch:1, batch:1835/2042, real_loss=0.601, fake_loss=0.765, g_loss=0.811
Epoch:1, batch:1836/2042, real_loss=0.624, fake_loss=0.669, g_loss=0.809
Epoch:1, batch:1837/2042, real_loss=0.727, fake_loss=0.648, g_loss=0.752
Epoch:1, batch:1838/2042, real_loss=0.545, fake_loss=0.644, g_loss=0.729
Epoch:1, batch:1839/2042, real_loss=0.553, fake_loss=0.972, g_loss=0.772
Epoch:1, batch:1840/2042, real_loss=0.538, fake_loss=0.726, g_loss=0.773
Epoch:1, batch:1841/2042, real_loss=0.731, fake_loss=0.836, g_loss=0.831
Epoch:1, batch:1842/2042, real_loss=0.829, fake_loss=0.760, g_loss=0.753
Epoch:1, batch:1843/2042, real_loss=0.512, fake_loss=0.826, g_loss=0.770
Epoch:1, batch:1844/2042, real_loss=0.597, fake_los

Epoch:1, batch:1945/2042, real_loss=0.739, fake_loss=0.689, g_loss=0.696
Epoch:1, batch:1946/2042, real_loss=0.742, fake_loss=0.574, g_loss=0.693
Epoch:1, batch:1947/2042, real_loss=0.700, fake_loss=0.770, g_loss=0.695
Epoch:1, batch:1948/2042, real_loss=0.687, fake_loss=0.624, g_loss=0.685
Epoch:1, batch:1949/2042, real_loss=0.718, fake_loss=0.690, g_loss=0.702
Epoch:1, batch:1950/2042, real_loss=0.685, fake_loss=0.704, g_loss=0.720
Epoch:1, batch:1951/2042, real_loss=0.719, fake_loss=0.677, g_loss=0.741
Epoch:1, batch:1952/2042, real_loss=0.702, fake_loss=0.676, g_loss=0.747
Epoch:1, batch:1953/2042, real_loss=0.720, fake_loss=0.722, g_loss=0.747
Epoch:1, batch:1954/2042, real_loss=0.659, fake_loss=0.681, g_loss=0.795
Epoch:1, batch:1955/2042, real_loss=0.737, fake_loss=0.666, g_loss=0.796
Epoch:1, batch:1956/2042, real_loss=0.605, fake_loss=0.677, g_loss=0.798
Epoch:1, batch:1957/2042, real_loss=0.577, fake_loss=0.702, g_loss=0.781
Epoch:1, batch:1958/2042, real_loss=0.751, fake_los

In [123]:
generator.predict([X_high[-1:], X_low[-1:], X_ele[-1:], X_other[-1:]])

array([[[[0.9136209 , 0.9154226 , 0.9151484 , 0.9143434 , 0.9116467 ],
         [0.9086517 , 0.9140768 , 0.91641533, 0.9136687 , 0.91142   ],
         [0.9146868 , 0.9137021 , 0.91137654, 0.9107744 , 0.9131136 ],
         [0.9146499 , 0.9150611 , 0.9134402 , 0.9144473 , 0.918668  ],
         [0.9128142 , 0.9168976 , 0.9175808 , 0.91791904, 0.91345584]],

        [[0.9101018 , 0.91260386, 0.91557777, 0.9159254 , 0.91106766],
         [0.9111054 , 0.9134025 , 0.912336  , 0.9159357 , 0.9133432 ],
         [0.915816  , 0.91321445, 0.9141924 , 0.9150695 , 0.9149438 ],
         [0.91476375, 0.9178445 , 0.91391087, 0.9157137 , 0.91906273],
         [0.9183154 , 0.91670185, 0.91325426, 0.91450804, 0.9148205 ]],

        [[0.9203325 , 0.917183  , 0.9172453 , 0.92293096, 0.9199325 ],
         [0.9171404 , 0.92018074, 0.91819096, 0.91595525, 0.9169756 ],
         [0.9211156 , 0.91926557, 0.9164213 , 0.91656387, 0.9199871 ],
         [0.9203504 , 0.9142437 , 0.91673577, 0.9156074 , 0.915655  ],
  

In [25]:
X = [X_high, X_low, X_ele, X_other]
j =0
batch_size=2
batch_X = [d[j*batch_size:(j+1)*batch_size] for d in X]

In [28]:
batch_X[0].shape

(2, 4, 5, 5, 1)

In [30]:
np.zeros((batch_size, 1), dtype='int')

array([[0],
       [0]])

# 3. downscaler test

In [215]:
reload(downscale)
from util_tools import downscale

In [216]:
dscler = downscale.downscaler(generator)

In [217]:
h_data = g_data[10:10+n_lag]
l_data = m_data[1836: 1856]
days = list(range(1836, 1856))

In [219]:
downscaled_data = dscler.downscale(h_data, l_data, ele_data,  [G_lats, G_lons, M_lats, M_lons], days, n_lag, n_pred, task_dim)

In [220]:
downscaled_data.shape

(16, 123, 207)

In [221]:
h_data.shape

(4, 123, 207)